In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")


In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='<think>\nOkay, so I need to translate "hi!" from English into Italian. Let\'s see... I know that in English, "hi" is a casual greeting, like saying hello to someone you\'re familiar with. In Italian, there are a few ways people greet each other. \n\nI remember that one common way is "ciao," but wait, doesn\'t that also mean goodbye? Hmm, maybe it\'s context-dependent. If I use "ciao" when meeting someone, it can be like saying hi, especially among friends or in informal settings. But I\'m not sure if that\'s the most appropriate translation for all situations.\n\nAnother option is "buongiorno," which means good morning or hello. That might be a safer bet because it\'s more formal and universally understood as a greeting. However, "buongiorno" can also be used to say goodbye in some contexts, so I need to make sure the tone matches.\n\nWait, there\'s also "salve." I think that\'s another way to say hello in Italian. It might be les

In [10]:
for token in model.stream(messages):
    print(token.content, end="|")

<think>|
|Okay|,| so| I| need| to| translate| "|hello|!"| from| English| into| Italian|.| Hmm|,| let| me| think| about| this|.| I| know| that| in| Italian|,| greetings| are| pretty| common|,| and| I|'ve| heard| people| say| "|c|iao|"| before|.| Is| that| right|?| But| wait|,| does| "|c|iao|"| mean| both| hello| and| goodbye|?| I| think| it| can| be| used| for| both| depending| on| the| context|.| So| if| someone| says| "|c|iao|!"| it|'s| like| a| casual| way| of| saying| either| hello| or| bye|.

|Is| there| another| word| for| hello| in| Italian| besides| "|c|iao|"?| Maybe| something| more| formal|?| I| believe| "|bu|ong|i|orno|"| is| also| a| greeting|,| but| that| specifically| means| good| morning|,| right|?| It| can| be| used| throughout| the| day| as| well|,| similar| to| "|hello|."| But| if| I| just| want| a| general| "|hello|,"| without| specifying| time| of| day|,| then| "|c|iao|"| would| probably| be| more| appropriate|.

|Let| me| double|-check|.| If| I| say| "|c|iao|!"| in|

In [11]:
idx = 0

async for event in model.astream_events(
    "Write me a 1 verse song about goldfish on the moon", version="v1"
):
    idx += 1
    if idx >= 5:  # Truncate the output
        print("...Truncated")
        break
    print(event)

{'event': 'on_chat_model_start', 'run_id': '44b3ae40-3a4f-4e7c-812b-6150f619536e', 'name': 'ChatOllama', 'tags': [], 'metadata': {}, 'data': {'input': 'Write me a 1 verse song about goldfish on the moon'}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '44b3ae40-3a4f-4e7c-812b-6150f619536e', 'tags': [], 'metadata': {}, 'name': 'ChatOllama', 'data': {'chunk': AIMessageChunk(content='<think>', additional_kwargs={}, response_metadata={}, id='run-44b3ae40-3a4f-4e7c-812b-6150f619536e')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '44b3ae40-3a4f-4e7c-812b-6150f619536e', 'tags': [], 'metadata': {}, 'name': 'ChatOllama', 'data': {'chunk': AIMessageChunk(content='\n', additional_kwargs={}, response_metadata={}, id='run-44b3ae40-3a4f-4e7c-812b-6150f619536e')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '44b3ae40-3a4f-4e7c-812b-6150f619536e', 'tags': [], 'metadata': {}, 'name': 'ChatOllama', 'data': {'chunk': AIMessageChunk(content='Okay', addi

In [12]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [13]:

from langchain_community.document_loaders import PyPDFLoader

file_path = "./data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


107


In [14]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [43]:
#langchain_core.documents.base.Document
import json

for split in all_splits:
        split_json = split.to_json()
        pretty_json = json.dumps(split_json['kwargs']['page_content'], indent=4)
        if '1967' in pretty_json:
                print(pretty_json)

"Table of Contents\nPART I\nITEM 1. BUSINESS\nGENERAL\nNIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this \"Annual Report\"), the terms \"we,\" \"us,\" \"our,\"\n\"NIKE\" and the \"Company\" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.\nOur principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is\nthe largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores\nand sales through our digital platforms (also referred to as \"NIKE Brand Digital\"), to retail accounts and to a mix of independent distributors, licensees and sales"


In [ ]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [18]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[-0.02281179, 0.071803115, -0.19200394, -0.06733745, 0.027059343, -0.024033826, 0.05242733, 0.010079667, 0.07523421, 0.0086905705]


In [20]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=all_splits)

In [21]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


In [ ]:
results = await vector_store.asimilarity_search("was incorporated?")

print(results[0])

page_content='Anti-takeover provisions may impair an acquisition of the Company or reduce the price of our common stock.
There are provisions within our articles of incorporation and Oregon law intended to protect shareholder interests by providing the Board of Directors a means to attempt
to deny coercive takeover attempts or to negotiate with a potential acquirer in order to obtain more favorable terms. Such provisions include a control share acquisition
statute, a freeze-out statute, two classes of stock that vote separately on certain issues, and the fact that holders of Class A Common Stock elect three-quarters of the
Board of Directors rounded down to the next whole number. However, such provisions could discourage, delay or prevent an unsolicited merger, acquisition or other
change in control of the Company that some shareholders might believe to be in their best interests or in which shareholders might receive a premium for their common' metadata={'producer': 'EDGRpdf Service w

In [45]:

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.8398329167006665

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

In [53]:
results = vector_store.similarity_search_with_score("When was Nike incorporated")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.71784198946112

page_content='transition of NIKE Brand businesses in certain countries within APLA to third-party distributors.
The Company's NIKE Direct operations are managed within each NIKE Brand geographic operating segment. Converse is also a reportable segment for the Company
and operates in one industry: the design, marketing, licensing and selling of athletic lifestyle sneakers, apparel and accessories.
Global Brand Divisions is included within the NIKE Brand for presentation purposes to align with the way management views the Company. Global Brand Divisions
revenues include NIKE Brand licensing and other miscellaneous revenues that are not part of a geographic operating segment. Global Brand Divisions costs represent
demand creation and operating overhead expense that include product creation and design expenses centrally managed for the NIKE Brand, as well as costs associated
with NIKE Direct global digital operations and enterprise technology.
(1)
2023 FORM 10-K 84

In [52]:

embedding = embeddings.embed_query("When was Nike incorporated")
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='transition of NIKE Brand businesses in certain countries within APLA to third-party distributors.
The Company's NIKE Direct operations are managed within each NIKE Brand geographic operating segment. Converse is also a reportable segment for the Company
and operates in one industry: the design, marketing, licensing and selling of athletic lifestyle sneakers, apparel and accessories.
Global Brand Divisions is included within the NIKE Brand for presentation purposes to align with the way management views the Company. Global Brand Divisions
revenues include NIKE Brand licensing and other miscellaneous revenues that are not part of a geographic operating segment. Global Brand Divisions costs represent
demand creation and operating overhead expense that include product creation and design expenses centrally managed for the NIKE Brand, as well as costs associated
with NIKE Direct global digital operations and enterprise technology.
(1)
2023 FORM 10-K 84' metadata={'producer': '

In [55]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=2)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='74dc2166-795a-4337-9906-d558201ed151', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further inf

# Classification Tutorial

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3.3", model_provider="ollama")

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = model.with_structured_output(Classification)



In [58]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [59]:
response.model_dump()

{'sentiment': 'positive', 'aggressiveness': 1, 'language': 'Spanish'}

In [61]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
    
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = model.with_structured_output(Classification)


In [62]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)


Classification(sentiment='positive', aggressiveness=0, language='Spanish')

In [63]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='negative', aggressiveness=8, language='Spanish')

In [64]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=0, language='english')

# Extracting data

In [88]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.
    # All results are string values so make sure to convert numbers to strings before returning

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[float] = Field(
        default=None, description="Height measured in meters"
    )

In [89]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
            "All results are strings so make sure to convert numbers to strings before returning",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [90]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3.3", model_provider="ollama")
#model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
structured_llm = model.with_structured_output(schema=Person)

In [91]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters=1.8288)